# Regression Data Set for ToyotaCorolla

## Data in-set and Preprocessing

In [247]:
import warnings
warnings.filterwarnings("ignore")

In [248]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso,LinearRegression,Ridge
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler,PolynomialFeatures, MinMaxScaler
from termcolor import colored
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error,accuracy_score,r2_score,recall_score,precision_score
from sklearn.svm import SVR,LinearSVR
from sklearn.pipeline import make_pipeline
import seaborn as sns
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostClassifier,AdaBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier

def prRed(skk): print("\033[1m \033[91m {}\033[00m" .format(skk)) 
def prGreen(skk): print("\033[1m \033[92m {}\033[00m" .format(skk)) 
def prYellow(skk): print("\033[1m \033[93m {}\033[00m" .format(skk)) 
def prBlack(skk): print("\033[1m \033[98m {}\033[00m" .format(skk)) 

In [249]:
corolla = pd.read_csv('ToyotaCorolla_kaggle.csv')
corolla.head()

,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,NaN,1,...,1,1,1,0,0,0,1,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,NaN,1,...,1,0,1,0,0,0,1,0,0,0
2,3,?TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90.0,1,...,0,0,1,0,0,0,1,0,0,0
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90.0,0,...,0,0,1,0,0,0,1,0,0,0
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90.0,0,...,1,1,1,0,1,0,1,0,0,0


In [250]:
corolla.drop(['Id','Model'], axis=1, inplace=True)
corolla.head()


,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,Automatic,cc,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,13500,23,10,2002,46986,Diesel,NaN,1,0,2000.0,...,1,1,1,0,0,0,1,0,0,0
1,13750,23,10,2002,72937,Diesel,NaN,1,0,2000.0,...,1,0,1,0,0,0,1,0,0,0
2,13950,24,9,2002,41711,Diesel,90.0,1,0,2000.0,...,0,0,1,0,0,0,1,0,0,0
3,14950,26,7,2002,48000,Diesel,90.0,0,0,2000.0,...,0,0,1,0,0,0,1,0,0,0
4,13750,30,3,2002,38500,Diesel,90.0,0,0,2000.0,...,1,1,1,0,1,0,1,0,0,0


##### NULL VALUES
We have null values in these columns - 'HP' , 'cc' , 'Doors', 'Cylinders', 'Gears', 'Weight'. )

In [251]:
corolla.isnull().sum()



Price                0
Age_08_04            0
Mfg_Month            0
Mfg_Year             0
KM                   0
Fuel_Type            0
HP                  70
Met_Color            0
Automatic            0
cc                  51
Doors               12
Cylinders           38
Gears               18
Quarterly_Tax        0
Weight               5
Mfr_Guarantee        0
BOVAG_Guarantee      0
Guarantee_Period     0
ABS                  0
Airbag_1             0
Airbag_2             0
Airco                0
Automatic_airco      0
Boardcomputer        0
CD_Player            0
Central_Lock         0
Powered_Windows      0
Power_Steering       0
Radio                0
Mistlamps            0
Sport_Model          0
Backseat_Divider     0
Metallic_Rim         0
Radio_cassette       0
Tow_Bar              0
dtype: int64

##### To impute null values, best categorical column to groupby would be 'Fuel_type'

In [252]:
g_corolla = corolla.groupby(['Fuel_Type'])
g_corolla.first()

,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
Fuel_Type,,,,,,,,,,,,,,,,,,,,,
CNG,7750,43,2,2001,178858,110.0,0,0,1600.0,3.0,...,0,0,1,1,0,0,1,0,1,0
Diesel,13500,23,10,2002,46986,90.0,1,0,2000.0,3.0,...,1,1,1,0,0,0,1,0,0,0
Petrol,21500,27,6,2002,19700,192.0,0,0,1800.0,3.0,...,1,1,1,1,0,0,0,1,1,0


In [253]:
g1_corolla = g_corolla.transform(lambda grp: grp.fillna(grp.mean()))

In [254]:
g1_corolla

,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,13500,23,10,2002,46986,78.306452,1,0,2000.0,3.0,...,1,1,1,0,0,0,1,0,0,0
1,13750,23,10,2002,72937,78.306452,1,0,2000.0,3.0,...,1,0,1,0,0,0,1,0,0,0
2,13950,24,9,2002,41711,90.000000,1,0,2000.0,3.0,...,0,0,1,0,0,0,1,0,0,0
3,14950,26,7,2002,48000,90.000000,0,0,2000.0,3.0,...,0,0,1,0,0,0,1,0,0,0
4,13750,30,3,2002,38500,90.000000,0,0,2000.0,3.0,...,1,1,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,7500,69,12,1998,20544,86.000000,1,0,1300.0,3.0,...,1,1,1,0,1,1,1,0,0,0
1432,10845,72,9,1998,19000,86.000000,0,0,1300.0,3.0,...,0,0,1,0,0,1,1,0,0,0
1433,8500,71,10,1998,17016,86.000000,0,0,1300.0,3.0,...,0,0,1,0,0,0,1,0,0,0
1434,7250,70,11,1998,16916,86.000000,1,0,1300.0,3.0,...,0,0,0,0,0,0,1,0,0,0


##### Group transformation with mean values in the above 6 columns

In [255]:
g1_corolla['HP']=(g_corolla['HP'].apply(lambda grp:grp.fillna(grp.mean())))

In [256]:
g1_corolla['cc']=(g_corolla['cc'].apply(lambda grp:grp.fillna(grp.mean())))


In [257]:
g1_corolla['Doors']=(g_corolla['Doors'].apply(lambda grp:grp.fillna(grp.median())))


In [258]:
g1_corolla['Cylinders']=(g_corolla['Cylinders'].apply(lambda grp:grp.fillna(grp.median())))


In [259]:
g1_corolla['Gears']=(g_corolla['Gears'].apply(lambda grp:grp.fillna(grp.median())))


In [260]:
g1_corolla['Weight']=(g_corolla['Weight'].apply(lambda grp:grp.fillna(grp.mean())))


In [261]:
g1_corolla= pd.concat([g1_corolla,corolla['Fuel_Type']], axis=1)


In [262]:
g1_corolla = pd.get_dummies(g1_corolla)

In [263]:
y1 = g1_corolla['Price']

In [264]:
g1_corolla.drop('Price', axis=1, inplace=True) #Since it's target variable

In [265]:
g1_corolla.head()

,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,Cylinders,...,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol
0,23,10,2002,46986,78.306452,1,0,2000.0,3.0,4.0,...,0,0,0,1,0,0,0,0,1,0
1,23,10,2002,72937,78.306452,1,0,2000.0,3.0,4.0,...,0,0,0,1,0,0,0,0,1,0
2,24,9,2002,41711,90.000000,1,0,2000.0,3.0,4.0,...,0,0,0,1,0,0,0,0,1,0
3,26,7,2002,48000,90.000000,0,0,2000.0,3.0,4.0,...,0,0,0,1,0,0,0,0,1,0
4,30,3,2002,38500,90.000000,0,0,2000.0,3.0,4.0,...,0,1,0,1,0,0,0,0,1,0


##### Selected Standardscaler because of difference in the values sizes and hence needed to be normalized.

In [266]:
X_train_org, X_test_org, y_train, y_test = train_test_split(g1_corolla,y1,test_size=0.25, random_state=0)

In [267]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train_org)
X_test = sc.transform(X_test_org)

In [268]:
X_train

array([[ 1.17930639, -0.75471747, -1.04764096, ..., -0.11053527,
        -0.34067899,  0.3623569 ],
       [-0.88746902,  0.13556809,  0.86582876, ..., -0.11053527,
        -0.34067899,  0.3623569 ],
       [-0.03956116, -1.05147933,  0.22800552, ..., -0.11053527,
         2.93531454, -2.75971013],
       ...,
       [ 0.75535246,  1.61937736, -1.04764096, ..., -0.11053527,
         2.93531454, -2.75971013],
       [-0.30453237,  0.43232994,  0.22800552, ..., -0.11053527,
        -0.34067899,  0.3623569 ],
       [ 0.64936398, -1.34824118, -0.40981772, ..., -0.11053527,
        -0.34067899,  0.3623569 ]])

In [269]:
X_train.shape

(1077, 36)

## ENSEMBLE MODELS

#### Bagging

##### With DecisionTreeRegressor

In [270]:
p_dt_reg = {'max_depth':[1,2,3,4,5]}
dt_reg = DecisionTreeRegressor(random_state=0)
grid_dt_reg = GridSearchCV(dt_reg,p_dt_reg,cv=3,iid='False')
grid_dt_reg.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=0, splitter='best'),
             iid='False', n_jobs=None,
             param_grid={'max_depth': [1, 2, 3, 4, 5]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=0)

In [271]:
print("Best max_depth :",grid_dt_reg.best_params_)

Best max_depth : {'max_depth': 5}


In [272]:
params = {'max_features': [2,5,10] ,'n_estimators': [100, 200, 300, 500],'max_samples': [0.1, 0.5, 1]}
dt_reg = DecisionTreeRegressor(max_depth=5,random_state=0)
bagging = BaggingRegressor(dt_reg,random_state=0)
grid_bag = GridSearchCV(bagging, params, cv=5,iid='False')

grid_bag.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=BaggingRegressor(base_estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                                             criterion='mse',
                                                                             max_depth=5,
                                                                             max_features=None,
                                                                             max_leaf_nodes=None,
                                                                             min_impurity_decrease=0.0,
                                                                             min_impurity_split=None,
                                                                             min_samples_leaf=1,
                                                                             min_samples_split=2,
                                                                             min_weight_fract

In [273]:
prRed("\nBagging wiht DT Regressor")
print("Best Params :", grid_bag.best_params_)
print("Training Score  -->", grid_bag.score(X_train,y_train))
print("Test Score      -->", grid_bag.score(X_test,y_test))

  
Bagging wiht DT Regressor
Best Params : {'max_features': 10, 'max_samples': 0.5, 'n_estimators': 100}
Training Score  --> 0.9048272785655762
Test Score      --> 0.8312525493300356


##### With SVR

In [274]:
# From Project 1, SVR best parameters were already determined as
# SVR -> Best parameters: {'C': 100, 'gamma': 0.001, 'kernel': 'linear'}

In [275]:
params = {'max_features': [2,5,10] ,'n_estimators': [100,200,300,500],'max_samples': [0.1,0.5,1]}
lsvr = SVR(C=100,gamma=0.001,kernel='linear')
bagging_lsvr = BaggingRegressor(lsvr,random_state=0)
grid_bag_lsvr = GridSearchCV(bagging_lsvr, params, cv=5,iid='False')

grid_bag_lsvr.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=BaggingRegressor(base_estimator=SVR(C=100,
                                                           cache_size=200,
                                                           coef0=0.0, degree=3,
                                                           epsilon=0.1,
                                                           gamma=0.001,
                                                           kernel='linear',
                                                           max_iter=-1,
                                                           shrinking=True,
                                                           tol=0.001,
                                                           verbose=False),
                                        bootstrap=True,
                                        bootstrap_features=False,
                                        max_features=1.0, max_samples=1.0,
                                  

In [276]:
prRed("\nBagging with Linear SVR")
print("Best Params :", grid_bag_lsvr.best_params_)
print("Training Score  -->", grid_bag_lsvr.score(X_train,y_train))
print("Test Score      -->", grid_bag_lsvr.score(X_test,y_test))

  
Bagging with Linear SVR
Best Params : {'max_features': 10, 'max_samples': 0.5, 'n_estimators': 100}
Training Score  --> 0.8043223862288902
Test Score      --> 0.7728180779522372


#### PASTING

##### With Decision Tree Regressor

In [277]:
params = {'max_features': [2,5,10] ,'n_estimators': [100, 200, 300, 500],'max_samples': [0.1, 0.5, 1]}
dt_reg = DecisionTreeRegressor(max_depth=5,random_state=0)
pasting = BaggingRegressor(dt_reg,random_state=0,bootstrap='False')
grid_pas = GridSearchCV(pasting, params, cv=5,iid='False')

grid_pas.fit(X_train,y_train)

prRed("\nPasting with DT Regressor")
print("Best Params :", grid_pas.best_params_)
print("Training Score  -->", grid_pas.score(X_train,y_train))
print("Test Score      -->", grid_pas.score(X_test,y_test))

  
Pasting with DT Regressor
Best Params : {'max_features': 10, 'max_samples': 0.5, 'n_estimators': 100}
Training Score  --> 0.9048272785655762
Test Score      --> 0.8312525493300356


##### With Linear SVR

In [278]:
params = {'max_features': [2,5,10] ,'n_estimators': [100,200,300,500],'max_samples': [0.1,0.5,1]}
lsvr = SVR(C=100,gamma=0.001,kernel='linear')
pasting_lsvr = BaggingRegressor(lsvr,random_state=0,bootstrap='False')
grid_pas_lsvr = GridSearchCV(pasting_lsvr, params, cv=5,iid='False')

grid_pas_lsvr.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=BaggingRegressor(base_estimator=SVR(C=100,
                                                           cache_size=200,
                                                           coef0=0.0, degree=3,
                                                           epsilon=0.1,
                                                           gamma=0.001,
                                                           kernel='linear',
                                                           max_iter=-1,
                                                           shrinking=True,
                                                           tol=0.001,
                                                           verbose=False),
                                        bootstrap='False',
                                        bootstrap_features=False,
                                        max_features=1.0, max_samples=1.0,
                               

In [279]:
prRed("\nPasting with Lasso")
print("Best Params :", grid_pas_lsvr.best_params_)
print("Training Score  -->", grid_pas_lsvr.score(X_train,y_train))
print("Test Score      -->", grid_pas_lsvr.score(X_test,y_test))

  
Pasting with Lasso
Best Params : {'max_features': 10, 'max_samples': 0.5, 'n_estimators': 100}
Training Score  --> 0.8043223862288902
Test Score      --> 0.7728180779522372


### ADABOOSTING REGRESSOR

#### - As we have already found the best parameters for SVR and Decision Tree Regressor, we will use the same parameters.

#### with DT Regressor

In [280]:
params = {'n_estimators': [50, 100, 200, 500],'learning_rate': [0.1, 0.25, 0.5, 0.75, 1]}
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5), random_state=0)

grid_ada = GridSearchCV(ada_reg, params, cv=5, iid='False')
grid_ada.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                                              criterion='mse',
                                                                              max_depth=5,
                                                                              max_features=None,
                                                                              max_leaf_nodes=None,
                                                                              min_impurity_decrease=0.0,
                                                                              min_impurity_split=None,
                                                                              min_samples_leaf=1,
                                                                              min_samples_split=2,
                                                                              min_we

In [281]:
prRed("\nAdaBoost Regressor with DT Regressor")
print("Best Params :", grid_ada.best_params_)
print("Training Score  -->", grid_ada.score(X_train,y_train))
print("Test Score      -->", grid_ada.score(X_test,y_test))

  
AdaBoost Regressor with DT Regressor
Best Params : {'learning_rate': 0.1, 'n_estimators': 500}
Training Score  --> 0.9508459766847791
Test Score      --> 0.8809075179420086


#### With SVR

In [282]:
params = {'n_estimators': [50, 100, 200, 500],'learning_rate': [0.1,  0.5,   1]}
ada_reg_svr = AdaBoostRegressor(SVR(C=100,gamma=0.001,kernel='linear'), random_state=0)

grid_ada_svr = GridSearchCV(ada_reg_svr, params, cv=5, iid='False')
grid_ada_svr.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=AdaBoostRegressor(base_estimator=SVR(C=100,
                                                            cache_size=200,
                                                            coef0=0.0, degree=3,
                                                            epsilon=0.1,
                                                            gamma=0.001,
                                                            kernel='linear',
                                                            max_iter=-1,
                                                            shrinking=True,
                                                            tol=0.001,
                                                            verbose=False),
                                         learning_rate=1.0, loss='linear',
                                         n_estimators=50, random_state=0),
             iid='False', n_jobs=None,
             param_grid={'learni

In [283]:
prRed("\nAdaBoost Regressor with SVR")
print("Best Params :", grid_ada_svr.best_params_)
print("Training Score  -->", grid_ada_svr.score(X_train,y_train))
print("Test Score      -->", grid_ada_svr.score(X_test,y_test))

  
AdaBoost Regressor with SVR
Best Params : {'learning_rate': 0.1, 'n_estimators': 50}
Training Score  --> 0.9211399353196834
Test Score      --> 0.8581108198426068


### GRADIENT BOOSTING

### With DT Reg

In [284]:
params = {'max_features': [2,5,10,20] ,'n_estimators': [50, 100, 200, 500],'learning_rate': [0.1, 0.25, 0.5, 0.75, 1]}
grad_gboost = GradientBoostingRegressor(random_state=0)

grid_gboost = GridSearchCV(grad_gboost,params,cv=5,iid='False')
grid_gboost.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_no_change=None,
                 

In [285]:
prRed("\nGradient Boosting with DT Regressor")
print("Best Params :", grid_gboost.best_params_)
print("Training Score  -->", grid_gboost.score(X_train,y_train))
print("Test Score      -->", grid_gboost.score(X_test,y_test))

  
Gradient Boosting with DT Regressor
Best Params : {'learning_rate': 0.1, 'max_features': 10, 'n_estimators': 100}
Training Score  --> 0.9553517920714709
Test Score      --> 0.8867838592519329


## Prinicipal Component Ananlysis

##### With 95% variance explained, Train and Test datasets were transformed.

In [286]:
from sklearn.decomposition import PCA
pca = PCA(.95)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [287]:
prRed("\nPost PCA Transformation :")
print("New dimension for the datasets is :",X_test_pca.shape[1])

  
Post PCA Transformation :
New dimension for the datasets is : 24


### <font color='red'> Models from Project 1 with PCA transformed 24 dimensions:
-Taking only grid searched optimum parameters and re executing all the models below.

#### Linear Regression 

In [288]:
g1_corolla.columns

Index(['Age_08_04', 'Mfg_Month', 'Mfg_Year', 'KM', 'HP', 'Met_Color',
       'Automatic', 'cc', 'Doors', 'Cylinders', 'Gears', 'Quarterly_Tax',
       'Weight', 'Mfr_Guarantee', 'BOVAG_Guarantee', 'Guarantee_Period', 'ABS',
       'Airbag_1', 'Airbag_2', 'Airco', 'Automatic_airco', 'Boardcomputer',
       'CD_Player', 'Central_Lock', 'Powered_Windows', 'Power_Steering',
       'Radio', 'Mistlamps', 'Sport_Model', 'Backseat_Divider', 'Metallic_Rim',
       'Radio_cassette', 'Tow_Bar', 'Fuel_Type_CNG', 'Fuel_Type_Diesel',
       'Fuel_Type_Petrol'],
      dtype='object')

In [289]:
lreg= LinearRegression()
lreg.fit(X_train_pca,y_train)
#lreg.fit(X_test_pca,y_test)
lreg_train_score_pca=lreg.score(X_train_pca,y_train)
lreg_test_score_pca=lreg.score(X_test_pca,y_test)
print("Linear Test Score -->", lreg_test_score_pca)

Linear Test Score --> 0.8332554899569575


#### Lasso Regression

In [290]:
lasso = Lasso(alpha=5)
lasso.fit(X_train_pca,y_train)
lasso_train_score_pca = lasso.score(X_train_pca,y_train)
lasso_test_score_pca = lasso.score(X_test_pca,y_test)
print("Lasso Test Score -->",lasso_test_score_pca)

Lasso Test Score --> 0.8339444787661593


#### Ridge Regression

In [291]:
ridge = Ridge(alpha = 15)
ridge.fit(X_train_pca,y_train)
ridge_train_score_pca=ridge.score(X_train_pca,y_train)
ridge_test_score_pca=ridge.score(X_test_pca,y_test)
print("Ridge Test Score -->",ridge_test_score_pca)

Ridge Test Score --> 0.8338818902061921


#### Polynomial Regression

In [292]:
poly = PolynomialFeatures(1)
poly.fit_transform(X_train_pca,y_train)
poly_train_score_pca=lreg.score(X_train_pca,y_train)
poly_test_score_pca=lreg.score(X_test_pca,y_test)
print("Poly Test Score -->",poly_test_score_pca)

Poly Test Score --> 0.8332554899569575


#### kNN Regressor

In [293]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_pca,y_train)
knn_train_score_pca=knn.score(X_train_pca,y_train)
knn_test_score_pca=knn.score(X_test_pca,y_test)
print("kNN Test Score -->",knn_test_score_pca)

kNN Test Score --> 0.8002188764797802


#### LinearSVR

In [294]:
param_grid = {'C':[10,50,100,150,200,250]}
lsvr = GridSearchCV(LinearSVR(), param_grid, cv=5, iid='False')
#print(lsvr.best_params_)
lsvr.fit(X_train_pca,y_train)
lsvr_train_score_pca=lsvr.score(X_train_pca,y_train)
lsvr_test_score_pca=lsvr.score(X_test_pca,y_test)
print("Linear SVR Test Score -->",lsvr_test_score_pca)

Linear SVR Test Score --> 0.8321815987158626


#### SVR

In [295]:
svr = SVR(C=100,gamma=0.001,kernel='linear')

svr.fit(X_train_pca,y_train)
svr_train_score_pca=svr.score(X_train_pca,y_train)
svr_test_score_pca=svr.score(X_test_pca,y_test)
print("SVR Test Score -->",svr_test_score_pca)

SVR Test Score --> 0.8330682575646479


In [296]:
PCA_score_model =  [('Linear ', lreg_train_score_pca, lreg_test_score_pca),
          ('Ridge ', ridge_train_score_pca, ridge_test_score_pca),
          ('Lasso ', lasso_train_score_pca, lasso_test_score_pca),
          ('Polynomial ', poly_train_score_pca, poly_test_score_pca),
          ('kNN ', knn_train_score_pca, knn_test_score_pca),
          ('LinearSVR ', lsvr_train_score_pca, lsvr_test_score_pca),
		  ('SVR ', svr_train_score_pca, svr_test_score_pca)]

In [297]:
stat = pd.DataFrame(data = PCA_score_model, columns = ['Models', 'PCA Train_Score', 'PCA Test_Score'])
prRed("With PCA")
print("\n",stat)

  With PCA

         Models  PCA Train_Score  PCA Test_Score
0      Linear          0.907934        0.833255
1       Ridge          0.907908        0.833882
2       Lasso          0.907888        0.833944
3  Polynomial          0.907934        0.833255
4         kNN          0.897985        0.800219
5   LinearSVR          0.904882        0.832182
6         SVR          0.904971        0.833068


### <font color='red'> Without PCA Transformation
             Models      Score_Train  Score_Test 
              Linear      0.922860    0.862248  
               Ridge      0.922295    0.860712  
               Lasso      0.922643    0.860758  
          Polynomial      0.922859    0.862262  
                 kNN      0.907167    0.818258  
                 SVM      0.917875    0.845226 
                 
### <font color='red'> After PCA Transformation
         Models  PCA Train_Score  PCA Test_Score
        Linear          0.907934      0.833255
         Ridge          0.907908      0.833882
         Lasso          0.907888      0.833944
    Polynomial          0.907934      0.833255
           kNN          0.897985      0.800219
     LinearSVR          0.904880      0.831845
           SVR          0.904971      0.833068

 - Test scores of models post PCA transformation are ~3%(average) lower than test scores without PCA transformation, which is unusual.

### NUERAL NETWORKS

#### Recurrent Neural Network

In [298]:
np.random.seed(10)
rnn = Sequential()
#Additon of input layer
rnn.add(Dense(36,input_dim=36,kernel_initializer='normal',activation='relu'))

#Addition of hidden layer
rnn.add(Dense(8,activation='relu',kernel_initializer='normal'))

#Output layer
rnn.add(Dense(1, kernel_initializer='normal'))

In [299]:
rnn.compile(loss='mse',optimizer='adam',metrics=['mse'])

In [300]:
rnn.fit(X_train,y_train,epochs=500, batch_size=20)

Epoch 1/500
1077/1077 [==============================] - 0s 161us/step - loss: 129959798.9229 - mse: 129959792.0000
Epoch 2/500
1077/1077 [==============================] - 0s 52us/step - loss: 129928688.3565 - mse: 129928664.0000
Epoch 3/500
1077/1077 [==============================] - 0s 65us/step - loss: 129753258.6890 - mse: 129753264.0000
Epoch 4/500
1077/1077 [==============================] - 0s 54us/step - loss: 129208048.2006 - mse: 129208040.0000
Epoch 5/500
1077/1077 [==============================] - 0s 54us/step - loss: 128049103.0938 - mse: 128049080.0000
Epoch 6/500
1077/1077 [==============================] - 0s 53us/step - loss: 126082507.4243 - mse: 126082520.0000
Epoch 7/500
1077/1077 [==============================] - 0s 69us/step - loss: 123142512.2674 - mse: 123142528.0000
Epoch 8/500
1077/1077 [==============================] - 0s 49us/step - loss: 119121168.4308 - mse: 119121184.0000
Epoch 9/500
1077/1077 [==============================] - 0s 49us/step - loss: 1

1077/1077 [==============================] - 0s 51us/step - loss: 1307964.4801 - mse: 1307964.3750
Epoch 146/500
1077/1077 [==============================] - 0s 49us/step - loss: 1296331.3058 - mse: 1296331.3750
Epoch 147/500
1077/1077 [==============================] - 0s 55us/step - loss: 1289107.5014 - mse: 1289107.2500
Epoch 148/500
1077/1077 [==============================] - 0s 56us/step - loss: 1278882.4776 - mse: 1278882.6250
Epoch 149/500
1077/1077 [==============================] - 0s 53us/step - loss: 1270457.9271 - mse: 1270457.5000
Epoch 150/500
1077/1077 [==============================] - 0s 51us/step - loss: 1261769.4240 - mse: 1261769.6250
Epoch 151/500
1077/1077 [==============================] - ETA: 0s - loss: 1193427.7500 - mse: 1193427.750 - 0s 50us/step - loss: 1254681.9615 - mse: 1254681.8750
Epoch 152/500
1077/1077 [==============================] - 0s 52us/step - loss: 1246175.8092 - mse: 1246175.6250
Epoch 153/500
1077/1077 [==============================] - 0

1077/1077 [==============================] - 0s 54us/step - loss: 836054.3715 - mse: 836054.3125
Epoch 290/500
1077/1077 [==============================] - 0s 50us/step - loss: 831709.5763 - mse: 831709.6250
Epoch 291/500
1077/1077 [==============================] - 0s 51us/step - loss: 831637.7170 - mse: 831637.7500
Epoch 292/500
1077/1077 [==============================] - 0s 54us/step - loss: 828573.7129 - mse: 828573.8125: 0s - loss: 812285.6250 - mse: 812285.812
Epoch 293/500
1077/1077 [==============================] - 0s 64us/step - loss: 827147.1552 - mse: 827147.2500
Epoch 294/500
1077/1077 [==============================] - 0s 50us/step - loss: 825304.2516 - mse: 825304.3750
Epoch 295/500
1077/1077 [==============================] - 0s 49us/step - loss: 825175.8125 - mse: 825175.8750
Epoch 296/500
1077/1077 [==============================] - 0s 52us/step - loss: 823206.7013 - mse: 823206.7500
Epoch 297/500
1077/1077 [==============================] - 0s 56us/step - loss: 8232

1077/1077 [==============================] - 0s 52us/step - loss: 715813.5263 - mse: 715813.5625
Epoch 436/500
1077/1077 [==============================] - 0s 54us/step - loss: 714895.1681 - mse: 714895.2500
Epoch 437/500
1077/1077 [==============================] - 0s 77us/step - loss: 713538.1630 - mse: 713538.0000
Epoch 438/500
1077/1077 [==============================] - 0s 84us/step - loss: 713207.5381 - mse: 713207.5625
Epoch 439/500
1077/1077 [==============================] - 0s 49us/step - loss: 713619.6438 - mse: 713619.6250
Epoch 440/500
1077/1077 [==============================] - 0s 52us/step - loss: 712752.8212 - mse: 712752.7500
Epoch 441/500
1077/1077 [==============================] - 0s 51us/step - loss: 710869.0070 - mse: 710868.9375
Epoch 442/500
1077/1077 [==============================] - 0s 52us/step - loss: 710349.0038 - mse: 710349.0000
Epoch 443/500
1077/1077 [==============================] - 0s 54us/step - loss: 711198.6260 - mse: 711198.5000
Epoch 444/500
1

In [301]:
#Model Evaluation

rnn.evaluate(X_test,y_test)

y_train_pred = rnn.predict(X_train)
y_test_pred = rnn.predict(X_test)

prRed("\nRNN Scores :")
print("Train Score -->", r2_score(y_train_pred,y_train))
print("Test Score  -->", r2_score(y_test_pred,y_test))



359/359 [==============================] - 0s 70us/step
  
RNN Scores :
Train Score --> 0.948001308610764
Test Score  --> 0.8661763993633015


#### Grid Search for RNN with best parameters 

In [302]:
def rnn_fn():
    
    rnn = Sequential()
    rnn.add(Dense(33,input_dim=36,activation='relu'))
    rnn.add(Dense(12,activation='relu'))
    rnn.add(Dense(1, activation = 'sigmoid'))
    
    rnn.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    return rnn


In [303]:
rnn_1 = KerasClassifier(build_fn = rnn_fn,verbose = 0)

params = {'batch_size':[10,15,20],'epochs':[100,250,500]}
rnn_grid = GridSearchCV(estimator=rnn_1 ,param_grid=params, cv=5,n_jobs=-1)

In [304]:
rnn_grid.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002561F924FC8>,
             iid='deprecated', n_jobs=-1,
             param_grid={'batch_size': [10, 15, 20], 'epochs': [100, 250, 500]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

#### RNN with the best parameters found -->

In [305]:
np.random.seed(10)
rnn = Sequential()
#Additon of input layer
rnn.add(Dense(36,input_dim=36,kernel_initializer='normal',activation='relu'))
rnn.add(Dense(8,activation='relu',kernel_initializer='normal'))
rnn.add(Dense(1, kernel_initializer='normal'))

In [306]:
rnn.compile(loss='mse',optimizer='adam',metrics=['mse'])

In [307]:
rnn.fit(X_train,y_train,epochs=500, batch_size=10)

Epoch 1/500
1077/1077 [==============================] - 0s 207us/step - loss: 129954573.2071 - mse: 129954568.0000
Epoch 2/500
1077/1077 [==============================] - 0s 107us/step - loss: 129757442.0724 - mse: 129757448.0000 0s - loss: 129819824.3168 - mse: 129819840.000
Epoch 3/500
1077/1077 [==============================] - 0s 108us/step - loss: 128732760.9582 - mse: 128732784.0000
Epoch 4/500
1077/1077 [==============================] - 0s 125us/step - loss: 126121796.2266 - mse: 126121792.0000
Epoch 5/500
1077/1077 [==============================] - 0s 159us/step - loss: 121334624.8691 - mse: 121334624.0000
Epoch 6/500
1077/1077 [==============================] - 0s 107us/step - loss: 114145620.8951 - mse: 114145600.0000
Epoch 7/500
1077/1077 [==============================] - 0s 109us/step - loss: 104707306.7409 - mse: 104707288.0000
Epoch 8/500
1077/1077 [==============================] - 0s 106us/step - loss: 93447994.6741 - mse: 93448000.0000
Epoch 9/500
1077/1077 [====

1077/1077 [==============================] - 0s 125us/step - loss: 952910.7739 - mse: 952911.0000
Epoch 145/500
1077/1077 [==============================] - 0s 110us/step - loss: 949245.7888 - mse: 949245.6875
Epoch 146/500
1077/1077 [==============================] - 0s 107us/step - loss: 944300.3243 - mse: 944300.2500
Epoch 147/500
1077/1077 [==============================] - 0s 109us/step - loss: 944812.1682 - mse: 944811.7500
Epoch 148/500
1077/1077 [==============================] - 0s 107us/step - loss: 938257.4884 - mse: 938257.3125
Epoch 149/500
1077/1077 [==============================] - 0s 108us/step - loss: 936442.7565 - mse: 936442.8750
Epoch 150/500
1077/1077 [==============================] - 0s 105us/step - loss: 934409.7794 - mse: 934409.6875
Epoch 151/500
1077/1077 [==============================] - 0s 107us/step - loss: 931453.7853 - mse: 931453.7500
Epoch 152/500
1077/1077 [==============================] - 0s 109us/step - loss: 928996.0831 - mse: 928996.1250
Epoch 

Epoch 290/500
1077/1077 [==============================] - 0s 107us/step - loss: 745496.8811 - mse: 745497.0000
Epoch 291/500
1077/1077 [==============================] - 0s 108us/step - loss: 746500.6000 - mse: 746500.4375
Epoch 292/500
1077/1077 [==============================] - 0s 140us/step - loss: 744619.9179 - mse: 744620.0000
Epoch 293/500
1077/1077 [==============================] - 0s 132us/step - loss: 743413.8267 - mse: 743413.8125
Epoch 294/500
1077/1077 [==============================] - 0s 131us/step - loss: 741514.1993 - mse: 741514.2500
Epoch 295/500
1077/1077 [==============================] - 0s 143us/step - loss: 742578.7214 - mse: 742578.5625
Epoch 296/500
1077/1077 [==============================] - 0s 168us/step - loss: 741476.5953 - mse: 741476.6250
Epoch 297/500
1077/1077 [==============================] - 0s 122us/step - loss: 739498.6693 - mse: 739498.8750
Epoch 298/500
1077/1077 [==============================] - 0s 277us/step - loss: 742331.0832 - mse: 7423

1077/1077 [==============================] - 0s 210us/step - loss: 667626.0697 - mse: 667626.0625
Epoch 436/500
1077/1077 [==============================] - 0s 208us/step - loss: 666995.7083 - mse: 666995.7500
Epoch 437/500
1077/1077 [==============================] - 0s 290us/step - loss: 665157.9343 - mse: 665158.0000
Epoch 438/500
1077/1077 [==============================] - 0s 162us/step - loss: 665577.7878 - mse: 665577.7500
Epoch 439/500
1077/1077 [==============================] - 0s 205us/step - loss: 664121.2649 - mse: 664121.3125
Epoch 440/500
1077/1077 [==============================] - 0s 144us/step - loss: 664215.8562 - mse: 664215.8750
Epoch 441/500
1077/1077 [==============================] - 0s 144us/step - loss: 663779.6170 - mse: 663779.5625
Epoch 442/500
1077/1077 [==============================] - 0s 209us/step - loss: 663401.6398 - mse: 663401.6875
Epoch 443/500
1077/1077 [==============================] - 0s 162us/step - loss: 661918.2576 - mse: 661918.3750
Epoch 

In [308]:
rnn.evaluate(X_test,y_test)

y_train_pred = rnn.predict(X_train)
y_test_pred = rnn.predict(X_test)

prRed("\nRNN Scores :")
print("Train Score -->", r2_score(y_train_pred,y_train))
print("Test Score  -->", r2_score(y_test_pred,y_test))

359/359 [==============================] - 0s 64us/step
  
RNN Scores :
Train Score --> 0.9511968672357237
Test Score  --> 0.8596356894515929


In [309]:
from IPython.display import display, Javascript
display(Javascript("""
require(
    ["base/js/dialog"], 
    function(dialog) {
        dialog.modal({
            title: 'Project_2_Regression Summary',
            body: 'Looking at post PCA scores, Linear regression has the highest score.',
            buttons: {
                'Best Model is yet again Linear!!': {}
            }
        });
    })
"""))

<IPython.core.display.Javascript object>